In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

url = 'https://www.nairaland.com/search?q=Chlamydia&search=Search'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
r = requests.get(url, headers = headers)
soup = BeautifulSoup(r.text, 'html.parser')


In [ ]:
full_list = []

for i in range(20):
  page_url = 'https://www.nairaland.com/search/chlamydia/0/0/0/'+ str(i)
  print(page_url)
  r = requests.get(page_url, headers = headers)
  soup = BeautifulSoup(r.text, 'html.parser')

  threads_on_page = soup.find_all('a',{'class':'user'})#[0].text
  print(len(threads_on_page))

  for j in range(len(threads_on_page)):
    temp_list=[]
    #username
    username = soup.find_all('a',{'class':'user'})[j].text
    print(username)
    
    #userURL
    userURL= 'https://www.nairaland.com/'+username
    print(userURL)

    r1 = requests.get(userURL, headers = headers)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    # thread_specifc_url= threads_on_page[j].find('a')['href']
    # print(thread_specifc_url)
    # r1 = requests.get(thread_specifc_url, headers = headers)
    # soup1 = BeautifulSoup(r1.text, 'html.parser')

    type = soup1.find_all('table')#[0].find_all('a')#{'class':'w'})#[0].text
    print(type)
    l= len(type)

    #profile_infor
    try:
      profile_infor= soup1.find_all('table')[l-3].find_all('p')#, "position" == 2)
      Personal_data_list= str(profile_infor[0]).replace('<p><b>', '||').replace('</b>', "").replace('<b>', '').replace('<p>', '||').split('||')

      #total_posts
      total_posts= (str(Personal_data_list[-1]).split('posts')[-1].split('<')[0]).replace('(','').replace(')','')

      temp_list=[username, userURL, Personal_data_list, total_posts]
      full_list.append(temp_list)
    except Exception as e:
      profile_infor= soup1.find_all('table')[l-2].find_all('p')#, "position" == 2)
      
      print('='*100)
      print(profile_infor)
      print('='*100)
      print(profile_infor[0])
      Personal_data_list= str(profile_infor[0]).replace('<p><b>', '||').replace('</b>', "").replace('<b>', '').replace('<p>', '||').split('||')

      #total_posts
      total_posts= (str(Personal_data_list[-1]).split('posts')[-1].split('<')[0]).replace('(','').replace(')','')
      #available_responses=int(row['total_available_responses'].replace('(','').replace(')',''))
    temp_list=[username, userURL, Personal_data_list, total_posts]
    full_list.append(temp_list)

df = pd.DataFrame(full_list,columns=("Profile Name","Profile URL", "Personal_data", "total_posts"))

In [31]:
df.shape

(683, 4)

In [33]:
df.head(15)

,Profile Name,Profile URL,Personal_data,total_posts
0,Faller,https://www.nairaland.com/Faller,"[, Location: London, Time registered: August 2...",672
1,Faller,https://www.nairaland.com/Faller,"[, Location: London, Time registered: August 2...",672
2,idrisaa951,https://www.nairaland.com/idrisaa951,"[, Personal text: Be Kind, Gender: m, Location...",5
3,MIsurgical,https://www.nairaland.com/MIsurgical,"[, Personal text: Minimal Access Surgery, Gend...",436
4,unclejoe2018,https://www.nairaland.com/unclejoe2018,"[, Personal text: Green Remedies Internation i...",37
5,Irisbaby,https://www.nairaland.com/Irisbaby,"[, Time registered: July 22, 2019, Time spent ...",357
6,Amandababy2,https://www.nairaland.com/Amandababy2,"[, Gender: f, Time registered: July 06, 2019, ...",376
7,Amandababy2,https://www.nairaland.com/Amandababy2,"[, Gender: f, Time registered: July 06, 2019, ...",376
8,MIsurgical,https://www.nairaland.com/MIsurgical,"[, Personal text: Minimal Access Surgery, Gend...",436
9,Irisbaby,https://www.nairaland.com/Irisbaby,"[, Time registered: July 22, 2019, Time spent ...",357


In [39]:
df['total_posts'].isnull().values.any()

False

In [34]:
df.to_csv('profile_extract_part1_Nigeria_hiv_part1.csv', index=False)

In [4]:
df= pd.read_csv('/content/drive/MyDrive/Colab Data/profile_extract_part1_Nigeria_hiv_part1.csv')

In [ ]:

full_final_list=[]
for index, row in df.iterrows():
    temp_list1 =[]
    print("***total_posts", row['total_posts']) 
    # if row[total_posts] is None:
      
    # available_responses=int(row['total_posts'])#.replace('(','').replace(')',''))
    if row['total_posts'] is None:
      post_url= 'NA'
      response_text= 'NA'
      response_time='NA'
      temp_list1=[row["Profile Name"], row["Profile URL"], row["Personal_data"], row["total_posts"], post_url, response_text, response_time]
      full_final_list.append(temp_list1)

    elif int(row['total_posts'])==0:
      
      post_url= 'NA'
      response_text= 'NA'
      response_time='NA'
      temp_list1=[row["Profile Name"], row["Profile URL"], row["Personal_data"], row["total_posts"], post_url, response_text, response_time]
      full_final_list.append(temp_list1)
    
    else:
      available_responses=int(row['total_posts'])
      user_url= row["Profile URL"]
      print(user_url)
      if available_responses<=30:
        z=1
      elif available_responses>30 and available_responses<=100:
        z=3
      elif available_responses>100 and available_responses<=300:
        z=10
      elif available_responses>300 and available_responses<=600:
        z=20
      elif available_responses>600:
        z=25
      #User_posts_url
       
      for n in range(z):
        user_posts_url= user_url+"/posts/"+ str(n)
        print('**')
        print(user_posts_url)
        print('**')
        r4 = requests.get(user_posts_url, headers = headers)
        soup4 = BeautifulSoup(r4.text, 'html.parser')
        
        #counting responses on current page
        responses_on_curr_page = soup4.find_all('div', {'class':"narrow"})
        print(len(responses_on_curr_page))
        for i in range(len(responses_on_curr_page)):
          
          #response_text
          response_text= responses_on_curr_page[i].text

          #response_time
          response_time_broken=soup4.find_all('span', {'class':"s"})[i].find_all('b')
          try:
            response_time= str(response_time_broken[0]).replace('<b>',"").replace('</b>',"")+' '+str(response_time_broken[1]).replace('<b>',"").replace('</b>',"")
          except Exception as e:
            response_time=response_time_broken
          #print(response_time)
          #print("="*100)
          temp_list1=[row["Profile Name"], row["Profile URL"], row["Personal_data"], row["total_posts"], user_posts_url, response_text, response_time]
          full_final_list.append(temp_list1)

df_final = pd.DataFrame(full_final_list,columns=("Profile Name","Profile URL", "Personal_data", "total_posts","user_posts_url", "response_text","response_time"))
print(df_final.shape)
df_final.to_csv('/content/drive/MyDrive/Colab Data/Nigeria_profile_extract_final.csv', index= False)

In [7]:
df_final.head()

,Profile Name,Profile URL,Personal_data,total_posts,user_posts_url,response_text,response_time
0,Faller,https://www.nairaland.com/Faller,"['', 'Location: London', 'Time registered: Aug...",672,https://www.nairaland.com/Faller/posts/0,See how the smoke black like Buhari's heart......,10:05pm Jun 21
1,Faller,https://www.nairaland.com/Faller,"['', 'Location: London', 'Time registered: Aug...",672,https://www.nairaland.com/Faller/posts/0,e,4:20pm Jun 21
2,Faller,https://www.nairaland.com/Faller,"['', 'Location: London', 'Time registered: Aug...",672,https://www.nairaland.com/Faller/posts/0,Na Dem go still use their hand bomb the place....,3:52pm Jun 21
3,Faller,https://www.nairaland.com/Faller,"['', 'Location: London', 'Time registered: Aug...",672,https://www.nairaland.com/Faller/posts/0,Kobicove,3:46pm Jun 21
4,Faller,https://www.nairaland.com/Faller,"['', 'Location: London', 'Time registered: Aug...",672,https://www.nairaland.com/Faller/posts/0,Catarrh abi?? I know wetin dey cause amYou do...,3:40pm Jun 21
